In [ ]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from IPython.display import clear_output
from tkinter import Tk, filedialog

from quick_pp.objects import Well, Project
import quick_pp.las_handler as las

In [ ]:
root = Tk()
file_objects = filedialog.askopenfiles(title='Choose well Log ASCII Standard (LAS) files to be combined',
                                        filetype=(('LAS Files', '*.LAS *.las'), ('All Files', '*.*')),
                                        multiple=True,
                                        mode='rb')
root.destroy()
if file_objects:
    project_name = "Mock"    
    project = Project(name=project_name)
    project.read_las([f.name for f in file_objects])
    project.save()

clear_output()

In [ ]:
# TODO: Rename and filter columns
import pandas as pd

df = pd.read_parquet(r"D:\ERMAI\data\03_primary\BEKK_CLEAN_1b8c72.parquet")
df.sort_values(['WELL_NAME', 'DEPTH'], inplace=True)

In [ ]:
marker_df = pd.read_csv(r"D:\quick_pp\notebooks\data\04_project\BEKK_MARKER.csv")

marker_df['WELL_NAME'] = marker_df['Well identifier (UWI)']
marker_df['DEPTH'] = marker_df['MDDF(ft)']
marker_df['ZONES'] = marker_df['Surface']
marker_df.sort_values(['WELL_NAME', 'DEPTH'], inplace=True)

In [ ]:
marker_df[marker_df.WELL_NAME == 'BEKK-005'].tail(20)

In [ ]:
df[df.WELL_NAME == 'BEKK-005'].tail()

In [ ]:
for well, data in df.groupby('WELL_NAME'):
    temp_df = marker_df[marker_df.WELL_NAME == well][['WELL_NAME', 'DEPTH', 'ZONES']].copy().reset_index(drop=True)
    df.loc[df.WELL_NAME == well, 'ZONES'] = pd.merge_asof(
        data['DEPTH'], temp_df, on='DEPTH', direction='forward'
    )['ZONES']
    # print('####', well, df.loc[df.WELL_NAME == well, 'ZONES'].isna().all())
    # print('zones \n', pd.merge_asof(
    #     data['DEPTH'], marker_df[marker_df.WELL_NAME == well], on='DEPTH', direction='forward',
    # )[['ZONES']])
    # break

In [ ]:
df[(df.WELL_NAME == 'BEKK-005') & df.ZONES.notna()].tail(50)

In [ ]:
df.columns

In [ ]:
df.loc[df.WELL_NAME == well, 'ZONES']

In [ ]:
# Loading a dataframe

root = Tk()
file_objects = filedialog.askopenfiles(title='Choose well Log ASCII Standard (LAS) files to be combined',
                                        filetype=(('LAS Files', '*.LAS *.las'), ('All Files', '*.*')),
                                        multiple=True,
                                        mode='rb')
root.destroy()
df, _ = las.read_las_files(file_objects)